In [1]:
import redis

# Configuration for the Redis connection
redis_host = 'redis-service'  # Update with your Redis host
redis_port = 6379             # Default Redis port
redis_db = 0                  # Redis database number
stream_name = 'WATERBODIES'           # Name of the Redis stream
consumer_group = 'my-consumer-group'  # Consumer group name
consumer_name = 'my-consumer'         # Unique name for the consumer

# Connect to Redis
r = redis.Redis(host=redis_host, port=redis_port, db=redis_db)

# Create a consumer group if it doesn't already exist
try:
    r.xgroup_create(stream_name, consumer_group, id='0', mkstream=True)
except redis.exceptions.ResponseError as e:
    # Ignore the error if the group already exists
    if "BUSYGROUP Consumer Group name already exists" not in str(e):
        raise

# Read messages from the stream
def read_stream():
    try:
        # Fetch new messages from the consumer group
        messages = r.xreadgroup(consumer_group, consumer_name, {stream_name: '>'}, count=10, block=5000)
        if messages:
            for stream, events in messages:
                for event_id, event_data in events:
                    print(f"Event ID: {event_id}")
                    print(f"Event Data: {event_data}")
                    # Acknowledge the message
                    r.xack(stream_name, consumer_group, event_id)
        else:
            print("No new messages")
    except Exception as e:
        print(f"Error reading stream: {e}")

# Continuously read from the stream
while True:
    read_stream()


No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages
Event ID: b'1729325410259-0'
Event Data: {b'message': b'{"message": "Workflow water-bodies-detection-msc8q has completed with status Succeeded", "item": "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20210708_0_L2A", "stac-catalog": "s3://results/water-bodies-detection-msc8q-bb26591e-d919-4582-87ea-9e4d8d4778cc/catalog.json"}'}
No new messages
No new messages
No new messages
No new messages
No new messages
No new messages


KeyboardInterrupt: 